In [1]:
import pandas as pd

#read in 538 SPI data (needs to scheduled after BPL matchday is completed)
teams = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv")
#filtering on Barclays Premier League teams only
select_league='Barclays Premier League'
drop=['Fulham','West Bromwich Albion','Sheffield United']
add=['Brentford','Watford','Norwich City']

Base_BPL=teams[(teams.league==select_league)& ~teams.name.isin(drop)]
Relegated=teams[teams.name.isin(drop)]
Promoted=teams[teams.name.isin(add)]
Promoted["league"].replace({"English League Championship": "Barclays Premier League"}, inplace=True)
New_BPL=Base_BPL.append(Relegated)

/Users/EddyZulkifly/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [2]:
New_BPL

,rank,prev_rank,name,league,off,def,spi
0,1,1,Manchester City,Barclays Premier League,2.79,0.25,92.42
2,3,3,Chelsea,Barclays Premier League,2.42,0.20,90.53
4,5,5,Liverpool,Barclays Premier League,2.69,0.40,89.17
7,8,8,Manchester United,Barclays Premier League,2.41,0.46,85.53
12,13,13,Arsenal,Barclays Premier League,2.18,0.42,83.72
24,25,25,Brighton and Hove Albion,Barclays Premier League,1.97,0.51,78.71
25,26,26,Leicester City,Barclays Premier League,2.14,0.64,78.22
26,27,27,Tottenham Hotspur,Barclays Premier League,2.31,0.76,78.06
33,34,34,West Ham United,Barclays Premier League,2.18,0.76,76.00
35,36,36,Leeds United,Barclays Premier League,2.13,0.78,74.94


In [3]:
#configure names to fit with understat csv names
Name=New_BPL["name"]
Name.replace('\s+', '_',regex=True,inplace=True)
t1=Name.replace('Brighton_and_Hove_Albion','Brighton')
t2=t1.replace('Leeds_United','Leeds')
t3=t2.replace('Leicester_City','Leicester')
t4=t3.replace('Newcastle','Newcastle_United')
t5=t4.replace('Norwich_City','Norwich')
t6=t5.replace('Tottenham_Hotspur','Tottenham')
t7=t6.replace('West_Ham_United','West_Ham')
t8=t7.replace('Wolverhampton','Wolverhampton_Wanderers')
t9=t8.sort_values()
t9

12                     Arsenal
43                 Aston_Villa
24                    Brighton
67                     Burnley
2                      Chelsea
95              Crystal_Palace
42                     Everton
58                      Fulham
35                       Leeds
25                   Leicester
4                    Liverpool
0              Manchester_City
7            Manchester_United
60            Newcastle_United
143           Sheffield_United
65                 Southampton
26                   Tottenham
110       West_Bromwich_Albion
33                    West_Ham
57     Wolverhampton_Wanderers
Name: name, dtype: object

In [4]:
#reading in data and merging files
#define understat which reads csv
understat = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/understat/understat_"+t9.iloc[5]+".csv")
#define merge which is the variable that stores all data
merge=understat.drop(understat.index)
#creates column for team
merge['team']=""
#need to add a loop, within each loop, add team name for identification
#merge into new dataframe, repeat till all data pulled from repo

In [5]:
#need to check if data is good or not status code 200 or nah
import requests
for i in t9:
    check=requests.get("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/understat/understat_"+str(i)+".csv")
    if check.status_code == 200:
        understat = pd.read_csv("https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/understat/understat_"+str(i)+".csv")
        understat['team']=str(i)
        merge=merge.append(understat)
    else:
        continue

merge['Season']='20-21'
merge

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,team,Season
0,a,2.162870,0.126327,2.162870,0.126327,"{'att': 307, 'def': 14}","{'att': 425, 'def': 13}",11,0,3,...,2.8937,w,2020-09-12 11:30:00,1,0,0,3,2.036543,Arsenal,20-21
1,h,1.329020,2.063770,1.329020,2.063770,"{'att': 217, 'def': 17}","{'att': 420, 'def': 23}",16,4,2,...,0.8155,w,2020-09-19 19:00:00,1,0,0,3,-0.734750,Arsenal,20-21
2,a,1.178170,2.738340,1.178170,2.738340,"{'att': 502, 'def': 14}","{'att': 256, 'def': 29}",3,11,1,...,0.3972,l,2020-09-28 19:15:00,0,0,1,0,-1.560170,Arsenal,20-21
3,h,0.667131,0.144994,0.667131,0.144994,"{'att': 167, 'def': 12}","{'att': 445, 'def': 23}",4,3,2,...,1.8925,w,2020-10-04 13:00:00,1,0,0,3,0.522137,Arsenal,20-21
4,a,0.838322,1.430230,0.838322,1.430230,"{'att': 346, 'def': 17}","{'att': 311, 'def': 23}",3,7,0,...,0.8436,l,2020-10-17 16:30:00,0,0,1,0,-0.591908,Arsenal,20-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,a,0.844138,1.071330,0.844138,1.071330,"{'att': 163, 'def': 20}","{'att': 224, 'def': 25}",8,5,1,...,1.0918,d,2021-05-03 17:00:00,0,1,0,1,-0.227192,Wolverhampton_Wanderers,20-21
34,h,1.119900,0.275710,1.119900,0.275710,"{'att': 193, 'def': 21}","{'att': 269, 'def': 22}",9,9,2,...,2.1303,w,2021-05-09 11:00:00,1,0,0,3,0.844190,Wolverhampton_Wanderers,20-21
35,a,1.092060,3.139470,1.092060,3.139470,"{'att': 369, 'def': 20}","{'att': 208, 'def': 18}",5,7,0,...,0.2744,l,2021-05-16 13:05:00,0,0,1,0,-2.047410,Wolverhampton_Wanderers,20-21
36,a,0.265526,0.670997,0.265526,0.670997,"{'att': 204, 'def': 23}","{'att': 312, 'def': 27}",1,4,0,...,0.8481,l,2021-05-19 17:00:00,0,0,1,0,-0.405471,Wolverhampton_Wanderers,20-21


In [6]:
merge.to_csv('2020-2021_Team_Understat.csv')